In [7]:
# %pip install nbformat

In [8]:
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

In [9]:
%run ml_project_normalized.ipynb

Note: you may need to restart the kernel to use updated packages.
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
Done running ml_project.ipynb.


In [10]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RandomizedSearchCV, cross_val_score, train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report
from imblearn.over_sampling import SMOTE, SMOTENC

In [11]:
from sklearn.compose import ColumnTransformer
from sklearn.svm import LinearSVC

def train_SVM_classifier(X_train, y_train, X_test):

    class_weights = get_class_weights(y_train)

    pipeline = Pipeline([
        ('smote', SMOTENC(categorical_features=[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11], random_state=42)),# 'intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed', 'is_mix', 'primary_color'], random_state=42)),
        ('freq', FunctionTransformer(apply_freq_encode, validate=False)),
        ('onehot', OneHotEncoder( handle_unknown='ignore', sparse_output=True)),
        ('linsvc', LinearSVC(class_weight=class_weights, max_iter=10000, dual=False, verbose=2))
    ])

    # Set up parameter distributions for XGBoost.
    param_distributions = {
    'linsvc__C': [0.01, 0.1, 1, 2, 4, 8, 10],
    'linsvc__loss': ['squared_hinge', 'hinge'],  # 'hinge' is okay too, but 'squared_hinge' usually works better
   # 'linsvc__penalty': ['l2'],  # 'l1' can only be used with dual=False and 'liblinear' solver
    'linsvc__max_iter': [1000, 5000, 10000],
    'linsvc__multi_class': ['ovr', 'crammer_singer'],  # try both strategies
    }
    # Perform hyperparameter search using RandomizedSearchCV.
    randomized_search = RandomizedSearchCV(
        estimator=pipeline, 
        param_distributions=param_distributions,
        n_iter=1,
        cv=skf, 
        scoring=balanced_acc_scorer, 
        verbose=3,
    )


        
    randomized_search.fit(X_train, y_train)
    
    print('Best parameters:', randomized_search.best_params_)
    print('Best cross-validation accuracy:', randomized_search.best_score_)
    
    cv_scores = cross_val_score(randomized_search.best_estimator_, X_train, y_train, cv=skf, verbose=3, scoring=balanced_acc_scorer)
    print('Generalization Balanced accuracy (via cross_val_score):', cv_scores.mean())

    # Make predictions on the test set using the best estimator.
    test_predictions = randomized_search.predict(X_test)
        
    return randomized_search.best_estimator_, test_predictions


In [13]:
# Cross Validation
X = df_train.drop(columns=['outcome_type'])
y = df_train['outcome_type']
X_train, X_test, y_train, y_test = train_test_split(X,y , test_size=0.20, random_state=42)


class_weights_cv = get_class_weights(y_train)

pipeline = Pipeline([
    ('smote', SMOTENC(categorical_features=[0, 1, 2, 3, 5, 6, 7, 8, 9, 10, 11], random_state=42)),# 'intake_type', 'intake_condition', 'animal_type', 'sex_upon_intake', 'breed', 'is_mix', 'primary_color'], random_state=42)),
    ('freq', FunctionTransformer(apply_freq_encode, validate=False)),
    ('onehot', OneHotEncoder( handle_unknown='ignore', sparse_output=True)),
    ('linsvc', LinearSVC(class_weight=class_weights_cv, max_iter=10000, dual=False, verbose=2))
])

# Set up parameter distributions for XGBoost.
param_distributions = {
    'linsvc__C': [0.01, 0.1, 1, 2, 4, 8, 10],
    'linsvc__loss': ['squared_hinge', 'hinge'],  # 'hinge' is okay too, but 'squared_hinge' usually works better
    # 'linsvc__penalty': ['l2'],  # 'l1' can only be used with dual=False and 'liblinear' solver
    'linsvc__max_iter': [1000, 5000, 10000],
    'linsvc__multi_class': ['ovr', 'crammer_singer'],  # try both strategies
}
# Perform hyperparameter search using RandomizedSearchCV.
randomized_search = RandomizedSearchCV(
    estimator=pipeline, 
    param_distributions=param_distributions,
    n_iter=1,
    cv=skf, 
    scoring=balanced_acc_scorer, 
    verbose=3,
)


    
randomized_search.fit(X_train, y_train)

print('Best parameters:', randomized_search.best_params_)
print('Best cross-validation accuracy:', randomized_search.best_score_)

cv_scores = cross_val_score(randomized_search.best_estimator_, X_train, y_train, cv=skf, verbose=3, scoring=balanced_acc_scorer)
print('Generalization Balanced accuracy (via cross_val_score):', cv_scores.mean())

# Make predictions on the test set using the best estimator.
test_predictions = randomized_search.predict(X_test)
classification_report_with_accuracy_score(y_test, test_predictions)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV 1/5] END linsvc__C=0.1, linsvc__loss=hinge, linsvc__max_iter=1000, linsvc__multi_class=ovr;, score=nan total time=   0.1s
[CV 2/5] END linsvc__C=0.1, linsvc__loss=hinge, linsvc__max_iter=1000, linsvc__multi_class=ovr;, score=nan total time=   0.1s
[CV 3/5] END linsvc__C=0.1, linsvc__loss=hinge, linsvc__max_iter=1000, linsvc__multi_class=ovr;, score=nan total time=   0.1s
[CV 4/5] END linsvc__C=0.1, linsvc__loss=hinge, linsvc__max_iter=1000, linsvc__multi_class=ovr;, score=nan total time=   0.1s
[CV 5/5] END linsvc__C=0.1, linsvc__loss=hinge, linsvc__max_iter=1000, linsvc__multi_class=ovr;, score=nan total time=   0.1s


ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "/u/nneoma/.local/lib/python3.8/site-packages/sklearn/utils/__init__.py", line 413, in _get_column_indices
    idx = _safe_indexing(np.arange(n_columns), key)
  File "/u/nneoma/.local/lib/python3.8/site-packages/sklearn/utils/__init__.py", line 355, in _safe_indexing
    return _array_indexing(X, indices, indices_dtype, axis=axis)
  File "/u/nneoma/.local/lib/python3.8/site-packages/sklearn/utils/__init__.py", line 184, in _array_indexing
    return array[key] if axis == 0 else array[:, key]
IndexError: index 9 is out of bounds for axis 0 with size 9

The above exception was the direct cause of the following exception:

Traceback (mos